In [2]:
#훈련 데이터 다운로드
from sklearn.datasets import fetch_20newsgroups
newsdata=fetch_20newsgroups(subset='train') #train을 넣으면 훈련 데이터, test를 넣으면 테스트 데이터를 다운로드
print(newsdata.keys()) #데이터 속성 출력

dict_keys(['data', 'filenames', 'target_names', 'target', 'DESCR'])


In [3]:
print (len(newsdata.data), len(newsdata.filenames), len(newsdata.target_names), len(newsdata.target)) #훈련용 샘플 개수

11314 11314 20 11314


In [4]:
print(newsdata.target_names) #20개의 데이터 카테고리 출력

['alt.atheism', 'comp.graphics', 'comp.os.ms-windows.misc', 'comp.sys.ibm.pc.hardware', 'comp.sys.mac.hardware', 'comp.windows.x', 'misc.forsale', 'rec.autos', 'rec.motorcycles', 'rec.sport.baseball', 'rec.sport.hockey', 'sci.crypt', 'sci.electronics', 'sci.med', 'sci.space', 'soc.religion.christian', 'talk.politics.guns', 'talk.politics.mideast', 'talk.politics.misc', 'talk.religion.misc']


In [5]:
print(newsdata.target[10]) #열한번째 샘플의 카테고리 번호

8


In [6]:
print(newsdata.target_names[8]) #8번 카테고리 제목

rec.motorcycles


In [7]:
print(newsdata.data[10]) #열한번째 샘플의 내용

From: irwin@cmptrc.lonestar.org (Irwin Arnstein)
Subject: Re: Recommendation on Duc
Summary: What's it worth?
Distribution: usa
Expires: Sat, 1 May 1993 05:00:00 GMT
Organization: CompuTrac Inc., Richardson TX
Keywords: Ducati, GTS, How much? 
Lines: 13

I have a line on a Ducati 900GTS 1978 model with 17k on the clock.  Runs
very well, paint is the bronze/brown/orange faded out, leaks a bit of oil
and pops out of 1st with hard accel.  The shop will fix trans and oil 
leak.  They sold the bike to the 1 and only owner.  They want $3495, and
I am thinking more like $3K.  Any opinions out there?  Please email me.
Thanks.  It would be a nice stable mate to the Beemer.  Then I'll get
a jap bike and call myself Axis Motors!

-- 
-----------------------------------------------------------------------
"Tuba" (Irwin)      "I honk therefore I am"     CompuTrac-Richardson,Tx
irwin@cmptrc.lonestar.org    DoD #0826          (R75/6)
-------------------------------------------------------------------

In [8]:
from sklearn.feature_extraction.text import CountVectorizer #입력한 텍스트를 자동으로 BoW로 만드는 CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.naive_bayes import MultinomialNB # 다항분포 나이브 베이즈 모델
from sklearn.metrics import accuracy_score #정확도 계산

In [9]:
dtmvector = CountVectorizer() #단어들의 카운트(출현 빈도)로 여러 문서들을 벡터화
X_train_dtm = dtmvector.fit_transform(newsdata.data)
print(X_train_dtm.shape) #(문서의 수, 훈련데이터의 단어 수)

(11314, 130107)


In [10]:
#TF-IDF 행렬 생성
tfidf_transformer = TfidfTransformer()  
tfidfv = tfidf_transformer.fit_transform(X_train_dtm)
print(tfidfv.shape)

(11314, 130107)


In [11]:
#모델 적용 (TF-IDF 행렬과 11,314개의 훈련 데이터에 대한 레이블이 적혀있는 newsdata.target)
mod = MultinomialNB()
mod.fit(tfidfv, newsdata.target)

MultinomialNB()

In [40]:
MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True) # alpha=1.0은 라플라스 스무딩

MultinomialNB()

In [12]:
newsdata_test = fetch_20newsgroups(subset='test', shuffle=True) #테스트 데이터 갖고오기
X_test_dtm = dtmvector.transform(newsdata_test.data) #테스트 데이터를 DTM으로 변환
tfidfv_test = tfidf_transformer.transform(X_test_dtm) #DTM을 TF-IDF 행렬로 변환
predicted = mod.predict(tfidfv_test) #테스트 데이터에 대한 예측
print("정확도:", accuracy_score(newsdata_test.target, predicted)) #예측값과 실제값 비교

정확도: 0.7738980350504514


In [21]:
#파이프라인 사용 -1번 naive_bayes
from sklearn.pipeline import Pipeline
text_clf = Pipeline([('vect', CountVectorizer()),
                     ('tfidf', TfidfTransformer()),
                     ('clf', MultinomialNB())])
text_clf2 = text_clf.fit(newsdata.data,newsdata.target)
predicted = text_clf2.predict(newsdata_test.data)
print("정확도:", accuracy_score(newsdata_test.target, predicted)) #예측값과 실제값 비교

정확도: 0.7738980350504514


In [35]:
#모델 변경을 위한 파이프라인 Classifier교체 -2번 svm
from sklearn.svm import LinearSVC
text_clf = Pipeline([('vect', CountVectorizer()),
                     ('tfidf', TfidfTransformer()),
                     ('clf', LinearSVC(C=10, loss='hinge', max_iter=2000, random_state=42))])
text_clf4 = text_clf.fit(newsdata.data,newsdata.target) 
predicted = text_clf4.predict(newsdata_test.data) 
print("정확도:", accuracy_score(newsdata_test.target, predicted)) #예측값과 실제값 비교

정확도: 0.848380244291025


In [36]:
#모델 변경을 위한 파이프라인 Classifier교체 -3번 sdg
from sklearn.linear_model import SGDClassifier
text_clf = Pipeline([('vect', CountVectorizer()),
                     ('tfidf', TfidfTransformer()),
                     ('clf', SGDClassifier(loss='hinge', alpha=1e-3, penalty='l2', random_state=42))])
text_clf3 = text_clf.fit(newsdata.data,newsdata.target) 
predicted = text_clf3.predict(newsdata_test.data) 
print("정확도:", accuracy_score(newsdata_test.target, predicted)) #예측값과 실제값 비교

정확도: 0.8240839086563994
